# MMVCのリポジトリをGoogle Driveに保存する

ver.2022/12/19

MMVC_Trainerのリポジトリと事前学習済みモデルを、Google Driveに保存します。

​

In [ ]:
#@title ## 1 Google Driveをマウント
#@markdown **このノートブックで、Google Driveを使用するための設定です。**

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#@markdown 　「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown 成功すれば、下記メッセージが出ます。 { display-mode: "form" }

#@markdown ``` 
#@markdown Mounted at /content/drive/
#@markdown ```

from google.colab import drive
drive.mount('/content/drive')

​

In [ ]:
#@title ## 2 MMVC_Trainerのクローン
#@markdown 以下で指定したディレクトリ上に、MMVC_Trainerというフォルダが作成されます。

#@markdown 実行前に、画面右上にあるアカウントのアイコンをクリックして、アカウントを確認してください。

#@markdown 成功すると、最後に以下のようなメッセージが表示されます。
#@markdown ```
#@markdown Checking out files: 100% (902/902), done.
#@markdown ```

#@markdown ​
#@markdown ### Settings
directory = "/content/drive/MyDrive" #@param {type:"string"}

%cd $directory
!git clone --depth=1 -b v1.3.3.0 https://github.com/stealthinu/MMVC_Trainer/

​

In [ ]:
#@title ## 3 fine_modelのダウンロード
#@markdown 事前学習済みのモデルをダウンロードします。

!curl -L https://dls.srgr0.com/D_180000.pth --output "{directory}/MMVC_Trainer/fine_model/D_180000.pth"
!curl -L https://dls.srgr0.com/G_180000.pth --output "{directory}/MMVC_Trainer/fine_model/G_180000.pth"

​

In [ ]:
#@title ## 4 確認
#@markdown 以下のようなメッセージが表示されれば、保存が正常に完了しています。
#@markdown ```
#@markdown Directory: MMVC_Trainer
#@markdown attentions.py
#@markdown commons.py
#@markdown ...(略)
#@markdown
#@markdown Directory: MMVC_Trainer/fine_model
#@markdown D_180000.pth
#@markdown G_180000.pth
#@markdown ```

print("Directory: MMVC_Trainer")
!ls -1 {directory}/MMVC_Trainer
print("\n")
print("Directory: MMVC_Trainer/fine_model")
!ls -1 {directory}/MMVC_Trainer/fine_model